In [1]:
import pandas as pd
import datetime
import time
from nba_api.stats.endpoints import LeagueDashPlayerStats

In [2]:
# measure type: usage
# per mode: per game
# 2021-22 start date: October 19, 2021
# today: November 24, 2021
# count n games

# 2020-21 start date: December 22, 2020
# find second date for n games

In [3]:
TODAY = datetime.date.today()
CURR_SEASON_START = datetime.datetime.strptime('2021-10-19','%Y-%m-%d').date()
PAST_SEASON_START = datetime.datetime.strptime('2020-12-22','%Y-%m-%d').date()
PAST_SEASON_CHECKPOINT = PAST_SEASON_START + (TODAY - CURR_SEASON_START)

CURR_SEASON = '2021-22'
PAST_SEASON = '2020-21'

COLS = ['PLAYER_NAME', 'TEAM_ABBREVIATION', 'GP', 'MIN', 'PCT_FTA']

In [4]:
CURR_SEASON_STATS = LeagueDashPlayerStats(
    measure_type_detailed_defense='Usage',
    per_mode_detailed='PerGame',
    date_from_nullable=CURR_SEASON_START,
    date_to_nullable=TODAY
).get_data_frames()[0][COLS]

time.sleep(10)

In [5]:
PAST_SEASON_STATS = LeagueDashPlayerStats(
    measure_type_detailed_defense='Usage',
    per_mode_detailed='PerGame',
    date_from_nullable=PAST_SEASON_START,
    date_to_nullable=PAST_SEASON_CHECKPOINT,
    season=PAST_SEASON
).get_data_frames()[0][COLS]

time.sleep(10)

In [6]:
CURR_SEASON_FTA = LeagueDashPlayerStats(
    measure_type_detailed_defense='Base',
    per_mode_detailed='Per36',
    date_from_nullable=CURR_SEASON_START,
    date_to_nullable=TODAY,
    season=CURR_SEASON
).get_data_frames()[0][['PLAYER_NAME', 'TEAM_ABBREVIATION', 'GP', 'MIN', 'FTA']]

time.sleep(10)

In [7]:
PAST_SEASON_FTA = LeagueDashPlayerStats(
    measure_type_detailed_defense='Base',
    per_mode_detailed='Per36',
    date_from_nullable=PAST_SEASON_START,
    date_to_nullable=PAST_SEASON_CHECKPOINT,
    season=PAST_SEASON
).get_data_frames()[0][['PLAYER_NAME', 'TEAM_ABBREVIATION', 'GP', 'MIN', 'FTA']]


In [8]:
# min games 15
# min minutes
CURR_SEASON_STATS = CURR_SEASON_STATS[
    (CURR_SEASON_STATS['GP'] >= 10) &
    (CURR_SEASON_STATS['MIN'] >= 100)
].sort_values(by='PCT_FTA', ascending=False).iloc[:20]

PLAYERS_LIST = list(CURR_SEASON_STATS['PLAYER_NAME'])

In [9]:
PAST_SEASON_STATS = PAST_SEASON_STATS[
    PAST_SEASON_STATS['PLAYER_NAME'].isin(PLAYERS_LIST)
].sort_values(by='PCT_FTA', ascending=False)

PAST_SEASON_STATS

,PLAYER_NAME,TEAM_ABBREVIATION,GP,MIN,PCT_FTA
159,Giannis Antetokounmpo,MIL,19,626.0,0.604
224,Joel Embiid,PHI,16,506.0,0.593
439,Trae Young,ATL,19,652.0,0.510
221,Jimmy Butler,MIA,8,244.0,0.462
266,Kevin Durant,BKN,16,589.0,0.450
24,Bam Adebayo,MIA,18,596.0,0.444
101,DeMar DeRozan,SAS,19,632.0,0.438
182,Ja Morant,MEM,7,195.0,0.434
390,Rudy Gobert,UTA,20,599.0,0.404
200,James Harden,BKN,16,609.0,0.401


In [10]:
CURR_SEASON_STATS

,PLAYER_NAME,TEAM_ABBREVIATION,GP,MIN,PCT_FTA
157,Giannis Antetokounmpo,MIL,20,648.0,0.674
230,Joel Embiid,PHI,10,323.0,0.613
224,Jimmy Butler,MIA,17,582.0,0.497
98,DeMar DeRozan,CHI,21,736.0,0.492
349,Nikola Jokic,DEN,14,454.0,0.484
390,Rudy Gobert,UTA,20,628.0,0.450
26,Bam Adebayo,MIA,17,557.0,0.444
183,Ja Morant,MEM,19,619.0,0.429
202,James Harden,BKN,20,705.0,0.426
284,Kristaps Porzingis,DAL,13,400.0,0.421


In [11]:
# combine data and begin graphing
CURR_SEASON_STATS.rename(columns={'PCT_FTA': 'CURR_FTR'}, inplace=True)
CURR_SEASON_STATS.drop(columns=['TEAM_ABBREVIATION', 'GP', 'MIN'], inplace=True)

In [12]:
df = CURR_SEASON_STATS.merge(PAST_SEASON_STATS, left_on='PLAYER_NAME', right_on='PLAYER_NAME')
df.rename(columns={'PCT_FTA': 'PAST_FTR'}, inplace=True)
df['FTR_DIFF'] = df['PAST_FTR'] - df['CURR_FTR']
df.drop(columns=['TEAM_ABBREVIATION', 'GP', 'MIN'], inplace=True)
df.sort_values(by='FTR_DIFF', ascending=False, inplace=True)

In [13]:
# filter FTA dfs
CURR_SEASON_FTA = CURR_SEASON_FTA[CURR_SEASON_FTA['PLAYER_NAME'].isin(PLAYERS_LIST)]
PAST_SEASON_FTA = PAST_SEASON_FTA[PAST_SEASON_FTA['PLAYER_NAME'].isin(PLAYERS_LIST)]

CURR_SEASON_FTA.rename(columns={'FTA': 'CURR_FTA'}, inplace=True)
PAST_SEASON_FTA.rename(columns={'FTA': 'PAST_FTA'}, inplace=True)

df = df.merge(CURR_SEASON_FTA[['PLAYER_NAME', 'CURR_FTA']], left_on='PLAYER_NAME', right_on='PLAYER_NAME')
df = df.merge(PAST_SEASON_FTA[['PLAYER_NAME', 'PAST_FTA']], left_on='PLAYER_NAME', right_on='PLAYER_NAME')

df['FTA_DIFF'] = df['PAST_FTA'] - df['CURR_FTA']


In [14]:
df

,PLAYER_NAME,CURR_FTR,PAST_FTR,FTR_DIFF,CURR_FTA,PAST_FTA,FTA_DIFF
0,Trae Young,0.410,0.510,0.100,6.1,11.2,5.1
1,Kevin Durant,0.411,0.450,0.039,6.9,8.2,1.3
2,Domantas Sabonis,0.373,0.383,0.010,4.6,5.6,1.0
3,Ja Morant,0.429,0.434,0.005,6.5,6.6,0.1
4,Bam Adebayo,0.444,0.444,0.000,7.0,6.5,-0.5
5,Anthony Davis,0.394,0.380,-0.014,6.4,6.4,0.0
6,Joel Embiid,0.613,0.593,-0.020,10.9,12.9,2.0
7,James Harden,0.426,0.401,-0.025,6.9,6.9,0.0
8,Jimmy Butler,0.497,0.462,-0.035,9.1,8.1,-1.0
9,Rudy Gobert,0.450,0.404,-0.046,7.5,6.5,-1.0


In [16]:
df.to_csv('data/nba_ftr_diffs.csv')